### Creating Interactive dashboard with the help of Dash and drawing graphs through Plotly. 
We will be building a Plotly Dash application for  to perform interactive visual analytics on SpaceX launch data in real-time.

This dashboard application contains input components such as a dropdown list and a range slider to interact with a pie chart and a scatter point chart. 



In [2]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
import plotly.express as px

### Importing Data and saving it into the dataframe.

In [3]:
url = "spacex_launch_dash.csv"
spacex_df = pd.read_csv(url)


In [ ]:
# Reading the  data into pandas dataframe
# spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Creating a dash application
app = dash.Dash(__name__)

# Creating an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                #  Adding a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown', options=[{'label':'All Sites', 'value':'All'}
                                ,{'label':'CCAFS LC-40','value':'CCAFS LC-40'}, 
                                {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},],
                                value="ALL", placeholder='Select a launch site form here',
                                searchable= True),
                                html.Br(),

                                # Adding a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, shows the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                #  Adding a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0, max=10000, step=1000,
                                marks={0:'0', 2500:'2500', 5000:'5000', 7500:'7500', 10000:'10000'}, 
                                value=[min_payload, max_payload]),

                                #   Adding a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])


 
# Adding a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'All':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Success Counts for all Sites')
        return fig
    else:
        new_df = spacex_df[spacex_df['Launch Site']==entered_site]['class'].value_counts().to_frame()
        new_df['name'] = ['Failure', 'Success']
        fig = px.pie(new_df, values='class',names='name',title='Total success Launches for site {}'.format(entered_site))
        return fig

# Adding a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id="payload-slider", component_property="value")])

def scatter(entered_site, payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0], payload[1])]

    if entered_site == 'All':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class',
                        color='Booster Version Category',
                        title="Success Count on Payload Mass for all sites")
        return fig
    else:
        fig = px.scatter(filtered_df[filtered_df['Launch Site'] == entered_site], x='Payload Mass (kg)', y='class',
                        color='Booster Version Category',
                        title="Success Count on Payload Mass for {}".format(entered_site))
        return fig   
         
# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)


 ![Title](Success_count.jpg)

#### We can see that the KSC LC-39A has the mose succesful launch record compared to other sites.

![Title](Success_on_payload.jpg)

#### Findings
We can see the the Booster version category FT is most successful<br>
We can also find that the successful payload category for all kind of Booster versions lies between 400 to 6000 Kg